In [1]:
 # в этой папке скрипт создаст (для каждой лекции из input) свою подпапку
output_dir = './output'
# в папке output, в подпапке каждой лекции лежат две папки: to_combine и to_print

# В этой папке лежат сделанные из pdf картинки страниц текста
# эта папка лежит в output_dir.
to_combine_dir = './to_combine'                   #В папке to_combine должны быть картинки ТОЛЬКО с цифрами: 1.png, 2.png и т.д.

# В этой папке лежат объединенные специфическим образом для печати на А4 картинки из папки to_combine
# Это и есть готовые конспекты к печати

to_print_dir = './to_print'


In [2]:
import os
import re
import shutil
from PIL import Image


In [3]:
def create_dir_if_it_doesnt_exist(root_dir, dir):
    path = os.path.join(root_dir, dir,)
        
    if not os.path.exists(path):
        os.makedirs(path)
        

In [4]:
image_height = 2339

In [5]:

def append_missing_images(output_dir, dirs):
    # Проходим по каждой подпапке в output_dir
    for input_filename in os.listdir(output_dir):
        input_path = os.path.join(output_dir, input_filename.strip())
        
        if os.path.isdir(input_path):
            # Проходим по папкам 'without_lines' и 'with_lines'
            for subdir in dirs:
                subdir_path = os.path.join(input_path, subdir)
                subdir_path = subdir_path.replace('./', '')

                if os.path.exists(subdir_path):
                    files = os.listdir(subdir_path)
                    
                    #for file in files:
                       # print(file)
                    
                    if len(files) == 0:
                        # Если файлов нет, пропускаем
                        print(f"Папка {subdir_path} пуста, пропускаем.")
                        continue
                    
                    # Если файлов больше 0 и не делится на 4
                    if len(files) > 0 and len(files) % 4 != 0:
                        # Вычисляем, сколько раз нужно добавить empty.png
                        while len(files) % 4 != 0 or (len(files) // 4) % 2 != 0:
                            print(f"Количество файлов в {subdir_path}: {len(files)}. Не кратно 4 или частное нечетное. Добавляем изображение.")
                            
                            # Путь к изображению empty.png
                            empty_image_path = './img/cells_half.png'
                            # Имя для нового изображения
                            new_image_name = f'{len(files) + 1}.png'
                            # Путь для копирования нового изображения
                            new_image_path = os.path.join(subdir_path, new_image_name)
                            
                            # Копируем empty.png в папку
                            shutil.copy(empty_image_path, new_image_path)
                            
                            # Добавляем новое изображение в список файлов
                            files.append(new_image_name)
                            
                        print(f"Файлы в {subdir_path} теперь кратны 4 и имеют четное частное.")
    

In [6]:

def combine_images(input_dir, output_dir, image_width, empty_dir):
    possesed_files=[]
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
   
    # Получаем список файлов в указанной директории
    files = sorted(os.listdir(input_dir), key=lambda x: int(re.search(r'\d+', x).group()))
    
    # Проходим по каждому второму файлу (четным индексам)
    for i in range(0, len(files), 1):
        #print(f'итерация: {i}, possesed files:')
        #print(possesed_files)
        #print('_____')
        
        #Если мы эти файлы обрабатывали, то скипаем
        #Пример: объединили 1ю и 3ю, 2ю и 4ю, дальше 3ю пропустили, 4ю пропустили, пошли ДАЛЬШЕ объединять 5 и 7, потом 6 и 8...
        if files[i] in possesed_files or files[i+2] in possesed_files:
            continue
        
        # Открываем текущие два изображения
        img1 = Image.open(os.path.join(input_dir, files[i])) #файл который сейчас обрабатываем. Допустим, 1.png
        img2 = None
        
        if i == len(files) - 1:
            print(f'i: {i}')
            
            if i % 2 != 0:
                
                img2 = Image.open(os.path.join(empty_dir)) #тогда это: 3.png
        else:             
            img2 = Image.open(os.path.join(input_dir, files[i + 2])) #тогда это: 3.png
            

            
        # Создаем новое изображение, горизонтально объединяя текущие два
        new_img = Image.new('RGBA', ( image_width , max(img1.height, img2.height)))

        # Проверяем, нужно ли перевернуть изображения на 180 градусов
        if i % 4 == 0:  # Для первой страницы (i=0, 4, 8, ...)
            img1 = img1.rotate(180)
        if i % 4 == 1:  # Для четвертой страницы (i=2, 6, 10, ...)
            img2 = img2.rotate(180)

        new_img.paste(img1, (0, 0), img1.convert('RGBA'))
                           
        new_img.paste(img2, ( int(image_width/2)+5, 0), img2.convert('RGBA'))



        # Создаем еще одно новое изображение, объединяя текущее с фоновым
        final_img = Image.new('RGBA', (new_img.width, max(new_img.height, image_height)))
        final_img.paste(new_img, (0, 0))


        # Сохраняем полученное изображение
        final_img.save(os.path.join(output_dir, f'{i+1}-{i+3}.png'))
              
        possesed_files += [files[i], files[i+2]]
        

    print("Изображения успешно объединены!") 
    

In [7]:
output_dir = './output'
image_width = 1660*2

In [ ]:
  
for dir__path in os.listdir(output_dir):
    full_path = os.path.join(output_dir, dir__path.replace('./','').strip())

    
    if os.path.isdir(full_path):
        for subdir__path in os.listdir(full_path):
            subdir_full_path = os.path.join(full_path, subdir__path)  # создаем полный путь

            to_combine_dir = to_combine_dir.replace('./', '').strip()
            to_print_dir = to_print_dir.replace('./', '').strip()

            # это не работает. Почему?
            if os.path.isdir(subdir_full_path):  # проверяем полный путь
                if to_combine_dir in subdir_full_path:
                    print(subdir_full_path) 
                    print(f'{output_dir}/{dir__path}/{to_print_dir}')
                    
                    append_missing_images(output_dir, [f'{to_combine_dir}'])
                    combine_images( f'{output_dir}/{dir__path}/{to_combine_dir}', f'{output_dir}/{dir__path}/{to_print_dir}', image_width, '../img/empty.png')
        